# Import Library

In [193]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import mysql.connector as mysql

# Konek Database

In [194]:
db_connection = mysql.connect(host = "10.4.19.15", 
                            database = 'mpninfo', 
                            user = 'indra', 
                            password= 'indra', 
                            port = 3306)

# READ FILE

In [195]:
mfwp = pd.read_csv('D:\DATA KANTOR\MASTERFILE\MFWP_AR_baru.csv',sep=';', dtype={'NPWP FULL':'str','Kode KLU':'str'}) 
map = pd.read_excel('D:\DATA KANTOR\MASTERFILE\KODE_MAP_FILTERED.xlsx', engine='openpyxl', dtype={'KD MAP':'str'})


# Kueri

In [196]:
query = '''
SELECT CONCAT(npwp,kpp,cabang) as Npwp_Full, nama, kdmap, kdbayar, datebayar, nominal,ntpn,bank,nosk,nospm FROM mpn
where date(datebayar) > '2019-12-31';
''' 

In [197]:
mpn = pd.read_sql(query, db_connection, parse_dates='datebayar')
mpn.head()

,Npwp_Full,nama,kdmap,kdbayar,datebayar,nominal,ntpn,bank,nosk,nospm
0,010030567007000,THE MASTER STEEL MANUFACTORY,411211,900,2021-05-17,32100897.0,72E7F83BAKB6UDB3,0009 - BANK NEGARA INDONESIA - MPN G2,None,None
1,010030567007000,THE MASTER STEEL MANUFACTORY,411211,900,2021-05-17,101713480.0,5E4D33L7RFF8TACS,0009 - BANK NEGARA INDONESIA - MPN G2,None,None
2,013412200007000,KARYAWAN WIJAYA KARYA,411211,900,2021-05-17,19600000.0,708B84HLBA2A0AU7,0009 - BANK NEGARA INDONESIA - MPN G2,None,None
3,013412200007000,KARYAWAN WIJAYA KARYA,411211,900,2021-05-17,4400000.0,84EF976TQPPF764S,0009 - BANK NEGARA INDONESIA - MPN G2,None,None
4,024586323009000,HARBARINDO BAHARITAMA,411211,900,2021-05-17,6586204.0,5DEC55E2R4MHSU12,0009 - BANK NEGARA INDONESIA - MPN G2,None,None


In [198]:
mpn.dtypes

Npwp_Full            object
nama                 object
kdmap                object
kdbayar              object
datebayar    datetime64[ns]
nominal             float64
ntpn                 object
bank                 object
nosk                 object
nospm                object
dtype: object

# + MFWP

In [199]:
mpn_ar = pd.merge(left=mpn, right=mfwp, how='left', left_on='Npwp_Full', right_on='NPWP FULL')
mpn_ar = mpn_ar.drop(['NPWP FULL','Nama WP'],axis=1)
mpn_ar.head(2)

,Npwp_Full,nama,kdmap,kdbayar,datebayar,nominal,ntpn,bank,nosk,nospm,NPWP9,Kode KLU,Status WP,Tgl.Daftar,NIP,Nama,Seksi
0,010030567007000,THE MASTER STEEL MANUFACTORY,411211,900,2021-05-17,32100897.0,72E7F83BAKB6UDB3,0009 - BANK NEGARA INDONESIA - MPN G2,None,None,10030567.0,24102,Aktif,31/12/1983,60105681.0,Donna Novita,Waskon 2
1,010030567007000,THE MASTER STEEL MANUFACTORY,411211,900,2021-05-17,101713480.0,5E4D33L7RFF8TACS,0009 - BANK NEGARA INDONESIA - MPN G2,None,None,10030567.0,24102,Aktif,31/12/1983,60105681.0,Donna Novita,Waskon 2


# + KDMAP

In [200]:
map.head()

,KD MAP,KODE MAP
0,411111,PPh Minyak Bumi
1,411112,PPh Gas Alam
2,411113,PPh Gas Alam
3,411119,PPh Migas Lainnya
4,411121,PPh 21


In [201]:
mpn_ar_map = pd.merge(left=mpn_ar, right=map,left_on='kdmap',right_on='KD MAP',how='left' )
mpn_ar_map.head(2)

,Npwp_Full,nama,kdmap,kdbayar,datebayar,nominal,ntpn,bank,nosk,nospm,NPWP9,Kode KLU,Status WP,Tgl.Daftar,NIP,Nama,Seksi,KD MAP,KODE MAP
0,010030567007000,THE MASTER STEEL MANUFACTORY,411211,900,2021-05-17,32100897.0,72E7F83BAKB6UDB3,0009 - BANK NEGARA INDONESIA - MPN G2,None,None,10030567.0,24102,Aktif,31/12/1983,60105681.0,Donna Novita,Waskon 2,411211,PPN Dalam Negeri
1,010030567007000,THE MASTER STEEL MANUFACTORY,411211,900,2021-05-17,101713480.0,5E4D33L7RFF8TACS,0009 - BANK NEGARA INDONESIA - MPN G2,None,None,10030567.0,24102,Aktif,31/12/1983,60105681.0,Donna Novita,Waskon 2,411211,PPN Dalam Negeri


In [202]:
mpn_ar_map = mpn_ar_map.rename(columns={'nama':'Wajib_Pajak','Nama':'AR'})
mpn_ar_map.head(2)

,Npwp_Full,Wajib_Pajak,kdmap,kdbayar,datebayar,nominal,ntpn,bank,nosk,nospm,NPWP9,Kode KLU,Status WP,Tgl.Daftar,NIP,AR,Seksi,KD MAP,KODE MAP
0,010030567007000,THE MASTER STEEL MANUFACTORY,411211,900,2021-05-17,32100897.0,72E7F83BAKB6UDB3,0009 - BANK NEGARA INDONESIA - MPN G2,None,None,10030567.0,24102,Aktif,31/12/1983,60105681.0,Donna Novita,Waskon 2,411211,PPN Dalam Negeri
1,010030567007000,THE MASTER STEEL MANUFACTORY,411211,900,2021-05-17,101713480.0,5E4D33L7RFF8TACS,0009 - BANK NEGARA INDONESIA - MPN G2,None,None,10030567.0,24102,Aktif,31/12/1983,60105681.0,Donna Novita,Waskon 2,411211,PPN Dalam Negeri


In [203]:
mpn_ar_map = mpn_ar_map[['Npwp_Full','Wajib_Pajak','KODE MAP','kdbayar','datebayar','nominal','ntpn','nosk','nospm','Kode KLU','Status WP','AR','Seksi']]

In [204]:
mpn_ar_map.head(2)

,Npwp_Full,Wajib_Pajak,KODE MAP,kdbayar,datebayar,nominal,ntpn,nosk,nospm,Kode KLU,Status WP,AR,Seksi
0,010030567007000,THE MASTER STEEL MANUFACTORY,PPN Dalam Negeri,900,2021-05-17,32100897.0,72E7F83BAKB6UDB3,None,None,24102,Aktif,Donna Novita,Waskon 2
1,010030567007000,THE MASTER STEEL MANUFACTORY,PPN Dalam Negeri,900,2021-05-17,101713480.0,5E4D33L7RFF8TACS,None,None,24102,Aktif,Donna Novita,Waskon 2


# +KLU

In [205]:
klu = pd.read_csv('D:/DATA KANTOR/MASTERFILE/KLU.csv', sep=';')
klu_kategori = pd.read_excel('D:/DATA KANTOR/MASTERFILE/KLU_sektor.xlsx', engine='openpyxl' )
print(klu.head())
klu_kategori.head()

  klu_kode                                           klu_nama klu_sektor
0    50131     ANGKUTAN LAUT DOMESTIK UMUM LINER UNTUK BARANG          H
1    47249                 PERDAGANGAN ECERAN MAKANAN LAINNYA          G
2    47413        PERDAGANGAN ECERAN PIRANTI LUNAK (SOFTWARE)          G
3    47863  PERDAGANGAN ECERAN KAKI LIMA DAN LOS PASAR BAR...          G
4    47891  PERDAGANGAN ECERAN KAKI LIMA DAN LOS PASAR HEW...          G


,Kode,Kategori
0,A,"Pertanian, Kehutanan dan Perikanan"
1,B,Pertambangan dan Penggalian
2,C,Industri Pengolahan
3,D,"Pengadaan Listrik, Gas, Uap/Air Panas dan Udar..."
4,E,"Pengadaan Air, Pengelolaan Sampah dan Daur Ula..."


In [206]:
mpn_ar_map_klu = pd.merge(left=mpn_ar_map, right=klu, left_on='Kode KLU', right_on='klu_kode', how='left')
mpn_ar_map_klu.head(2)

,Npwp_Full,Wajib_Pajak,KODE MAP,kdbayar,datebayar,nominal,ntpn,nosk,nospm,Kode KLU,Status WP,AR,Seksi,klu_kode,klu_nama,klu_sektor
0,010030567007000,THE MASTER STEEL MANUFACTORY,PPN Dalam Negeri,900,2021-05-17,32100897.0,72E7F83BAKB6UDB3,None,None,24102,Aktif,Donna Novita,Waskon 2,24102,INDUSTRI PENGGILINGAN BAJA (STEEL ROLLING),C
1,010030567007000,THE MASTER STEEL MANUFACTORY,PPN Dalam Negeri,900,2021-05-17,101713480.0,5E4D33L7RFF8TACS,None,None,24102,Aktif,Donna Novita,Waskon 2,24102,INDUSTRI PENGGILINGAN BAJA (STEEL ROLLING),C


In [207]:
mpn_ar_map_klu = pd.merge(left=mpn_ar_map_klu, right=klu_kategori, left_on='klu_sektor', right_on='Kode', how='left')
mpn_ar_map_klu.head(2)

,Npwp_Full,Wajib_Pajak,KODE MAP,kdbayar,datebayar,nominal,ntpn,nosk,nospm,Kode KLU,Status WP,AR,Seksi,klu_kode,klu_nama,klu_sektor,Kode,Kategori
0,010030567007000,THE MASTER STEEL MANUFACTORY,PPN Dalam Negeri,900,2021-05-17,32100897.0,72E7F83BAKB6UDB3,None,None,24102,Aktif,Donna Novita,Waskon 2,24102,INDUSTRI PENGGILINGAN BAJA (STEEL ROLLING),C,C,Industri Pengolahan
1,010030567007000,THE MASTER STEEL MANUFACTORY,PPN Dalam Negeri,900,2021-05-17,101713480.0,5E4D33L7RFF8TACS,None,None,24102,Aktif,Donna Novita,Waskon 2,24102,INDUSTRI PENGGILINGAN BAJA (STEEL ROLLING),C,C,Industri Pengolahan


In [208]:
mpn_ar_map_klu = mpn_ar_map_klu.drop(columns=['klu_kode','klu_sektor','Kode','nospm'])
mpn_ar_map_klu.head(2)

,Npwp_Full,Wajib_Pajak,KODE MAP,kdbayar,datebayar,nominal,ntpn,nosk,Kode KLU,Status WP,AR,Seksi,klu_nama,Kategori
0,010030567007000,THE MASTER STEEL MANUFACTORY,PPN Dalam Negeri,900,2021-05-17,32100897.0,72E7F83BAKB6UDB3,None,24102,Aktif,Donna Novita,Waskon 2,INDUSTRI PENGGILINGAN BAJA (STEEL ROLLING),Industri Pengolahan
1,010030567007000,THE MASTER STEEL MANUFACTORY,PPN Dalam Negeri,900,2021-05-17,101713480.0,5E4D33L7RFF8TACS,None,24102,Aktif,Donna Novita,Waskon 2,INDUSTRI PENGGILINGAN BAJA (STEEL ROLLING),Industri Pengolahan


In [209]:
header = pd.Series(mpn_ar_map_klu.columns)
header = header.str.upper()
header = header.str.replace(' ','_')

In [210]:
mpn_ar_map_klu.columns = header
mpn_ar_map_klu.head(2)

,NPWP_FULL,WAJIB_PAJAK,KODE_MAP,KDBAYAR,DATEBAYAR,NOMINAL,NTPN,NOSK,KODE_KLU,STATUS_WP,AR,SEKSI,KLU_NAMA,KATEGORI
0,010030567007000,THE MASTER STEEL MANUFACTORY,PPN Dalam Negeri,900,2021-05-17,32100897.0,72E7F83BAKB6UDB3,None,24102,Aktif,Donna Novita,Waskon 2,INDUSTRI PENGGILINGAN BAJA (STEEL ROLLING),Industri Pengolahan
1,010030567007000,THE MASTER STEEL MANUFACTORY,PPN Dalam Negeri,900,2021-05-17,101713480.0,5E4D33L7RFF8TACS,None,24102,Aktif,Donna Novita,Waskon 2,INDUSTRI PENGGILINGAN BAJA (STEEL ROLLING),Industri Pengolahan


In [211]:
mpn_ar_map_klu.to_excel('D:\DATA KANTOR\JUPYTER NOTEBOOK\Export\MPNSPM.xlsx',index=False)